In [1]:
import os
import time
import csv
import numpy as np
import pandas as pd

import gensim
import sklearn.metrics
import re
import unicodedata
from tqdm import tqdm
import sys
import hashlib
import random


In [2]:


import re
df = pd.read_csv("crawled_issue/output.csv")
df_unique = df.drop_duplicates(subset='Issue ID', keep='first')
df = df_unique
print(df.head())


     Issue ID                                         Issue Body
0  2032018852                   Changed "ar" to "are" in readme.
1  2036967098  ### Describe the Bug\n\numami behaving like a ...
2  2039165352  Closes #800, #2423, #2400, #2386, #2383, #2358...
3  2041048669  ### Describe the Bug\n\nImpossible to load dat...
4  2048399912  ### Describe the feature or enhancement\n\nWe ...


In [3]:
# !pip3 install openpyxl 
excel_data = pd.read_excel('dataset/Secret-Regular-Expression.xlsx')

# Read the values of the file in the dataframe
regex = pd.DataFrame(excel_data, columns=[
'Pattern_ID','Secret Type',	'Regular Expression','Source'])

In [4]:

data_dict={}
for j in df.index:
        # if df["id"][j] != "1165939311":
        #         continue
        input_string =    str(df["Issue Body"][j])    
        input_string = re.sub(r'[\'"\│]', '', input_string)
        dir_list_clean = re.sub(r'drwx[-\s]*\d+\s+\w+\s+\w+\s+\d+\s+\w+\s+\d+\s+[0-9a-fA-F-]+.*','',input_string)
        shell_code_free_text = re.sub(r'```shell([^`]+)```','',dir_list_clean,flags=re.IGNORECASE)
        shell_code_free_text = re.sub(r'```Shell\s*"([^"]*)"\s*```','',shell_code_free_text,flags=re.IGNORECASE)
        # saved_game_free_text = re.sub(r'```([^`]+)```','',shell_code_free_text) #etay jhamela hobe
        saved_game_free_text = re.sub(r'<details><summary>Saved game</summary>\n\n```(.*?)```', '', shell_code_free_text)
        remove_packages = re.sub(r'(\w+\.)+\w+','',saved_game_free_text)
        java_exp_free_text = re.sub(r'at\s[\w.$]+\.([\w]+)\(([^:]+:\d+)\)','',remove_packages)
        # url_free_text= re.sub(https?://[^\s#]+#[A-Za-z0-9\-]+,'', java_exp_free_text, flags=re.IGNORECASE)
        url_with_fragment_text= re.sub(r'https?://[^\s#]+#[A-Za-z0-9\-\=\+]+','', java_exp_free_text, flags=re.IGNORECASE)
        url_free_text= re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '',url_with_fragment_text)
        commit_free_text= re.sub(r'commit[ ]?(?:id)?[ ]?[:]?[ ]?([0-9a-f]{40})\b', '', url_free_text, flags=re.IGNORECASE)
        file_path_free_text = re.sub(r"/[\w/. :-]+",'',commit_free_text)
        file_path_free_text = re.sub( r'(/[^/\s]+)+','',file_path_free_text)
        sha256_free_text = re.sub(r'sha256\s*[:]?[=]?\s*[a-fA-F0-9]{64}','',file_path_free_text)
        sha1_free_text = re.sub(r'git-tree-sha1\s*=\s*[a-fA-F0-9]+','',sha256_free_text)
        build_id_free_text = re.sub(r'build-id\s*[:]?[=]?\s*([a-fA-F0-9]+)','',sha1_free_text)
        guids_free_text = re.sub(r'GUIDs:\s+([0-9a-fA-F-]+\s+[0-9a-fA-F-]+\s+[0-9a-fA-F-]+)','',build_id_free_text)
        uuids_free_text = re.sub(r'([0-9a-fA-F-]+\s*,\s*[0-9a-fA-F-]+\s*,\s*[0-9a-fA-F-]+)','',guids_free_text)
        event_id_free_text = re.sub(r'<([^>]+)>','',uuids_free_text)
        UUID_free_text = re.sub(r'(?:UUID|GUID|version|id)[\\=:"\'\s]*\b[a-fA-F0-9]{8}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{12}\b'
,'',event_id_free_text,flags=re.IGNORECASE) ##without the prefix so many false positives can be omitted
        hex_free_text = re.sub(r'(?:data|address|id)[\\=:"\'\s]*\b0x[0-9a-fA-F]+\b','',UUID_free_text,flags=re.IGNORECASE) ## deleting hex ids directly can cause issues
        ss_free_text = re.sub(r'Screenshot_(\d{4}[_-]\d{2}[_-]\d{2}[_-]\d{2}[_-]\d{2}[_-]\d{2}[_-]\d{2}[_-]\w+)','',hex_free_text,flags=re.IGNORECASE)
        cleaned_text = ss_free_text
        # file_path = "output.txt"

        # with open(file_path, 'w') as file:
        #                 file.write(cleaned_text)
        data_dict[j] = {'Issue ID':df['Issue ID'][j],'Issue Body':cleaned_text}
        # idx = idx+1
    


cleaned_text_data = pd.DataFrame.from_dict(data_dict, "index")
cleaned_text_data

,Issue ID,Issue Body
0,2032018852,Changed ar to are in readme.
1,2036967098,### Describe the Bug\n\numami behaving like a ...
2,2039165352,"Closes #800, #2423, #2400, #2386, #2383, #2358..."
3,2041048669,### Describe the Bug\n\nImpossible to load dat...
4,2048399912,### Describe the feature or enhancement\n\nWe ...
5,2053026441,fixed #2433 bug please review\r\n
6,2053308325,This PR includes updates to the Swedish transl...
7,2054244873,"### Describe the Bug\n\n1. Start onboarding, c..."
8,2056044123,### Describe the Bug\r\n\r\ni want to track re...
9,2058073803,This PR corrects a spelling error in the Share...


In [5]:
###recovered
    
idx = 0
data_dict={}
# start = iter*100000
# end = (iter+1)*100000
for i in regex.index:
    #print(i,regex['Secret Type'][i]) #, regex['Regular Expression'][i])
    # if i%100==0:
    #     print("checkpoint")
    p = re.compile(regex['Regular Expression'][i])
    
    # print("=====================================================================")
    
    for j in df.index:
        
        cleaned_text = cleaned_text_data.loc[j, 'Issue Body']
            # Now you can use 'cleaned_text' for further processing
       
        matches = re.findall(p,cleaned_text)
        for match in set(matches):
                data_dict[idx] = {'Type': regex['Secret Type'][i], 'Issue ID':df['Issue ID'][j],'Candidate String':match} #,'Entropy':shannon_entropy(match)}
                idx = idx+1
    


data = pd.DataFrame.from_dict(data_dict, "index")
print(data.shape)
data=data.drop_duplicates(subset=["Issue ID", "Candidate String"], keep='first')
data.to_csv('crawled_issue/issues-with-candidate-strings.csv')

C:\Users\ahmed\AppData\Local\Temp\ipykernel_17968\1642664489.py:11: FutureWarning: Possible nested set at position 35
  p = re.compile(regex['Regular Expression'][i])
C:\Users\ahmed\AppData\Local\Temp\ipykernel_17968\1642664489.py:11: DeprecationWarning: Flags not at the start of the expression '/(?i)-----\\s*?BEGIN[' (truncated) but at position 1
  p = re.compile(regex['Regular Expression'][i])


(42, 3)
